## PageRank

## Exercise 1

In [1]:
import numpy as np
from scipy.sparse import dok_matrix
from scipy import linalg as la
import pandas as pd

In [2]:
def readmatfromtext(filename):
    '''
    Accepts a .txt file with nodes listed in each line and
    creates an adjacency matrix from them.
    '''
    nodes = []
    N = 0
    with open(filename, 'r') as myfile:
        for line in myfile:
            try:
                line = list(map(int, line.strip().split()))
                nodes.append(line)
            except:
                pass
    #Grab highest node
    N = np.amax(nodes) + 1
    A = np.zeros((N,N))
    for i in range(N):
        for j in range(N):
            if[i, j] in nodes:
                A[i, j] = 1
    return dok_matrix(A)

readmatfromtext("Data/matrix.txt").toarray()

array([[0., 0., 0., 0., 0., 0., 0., 1.],
       [1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 1., 0., 0., 0., 1., 0.],
       [1., 0., 0., 0., 0., 1., 1., 0.],
       [1., 0., 0., 0., 0., 0., 1., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0.]])

## Exercise 2

In [13]:
def Kmat(A):
    '''
    Accepts an adjacency matrix and returns the K matrix
    '''
    #Compute modified adjacency matrix
    N = A.shape[0]
    A[A.sum(axis=1) == 0, :] = np.ones(N)
    # Get diagonals
    D = A.sum(axis = 1)
    K = A.T / D
    return(K)

Data = readmatfromtext("Data/matrix.txt").toarray()

Kmat(Data)

array([[0.        , 1.        , 0.125     , 0.33333333, 0.33333333,
        0.5       , 1.        , 1.        ],
       [0.        , 0.        , 0.125     , 0.        , 0.        ,
        0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.125     , 0.33333333, 0.        ,
        0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.125     , 0.        , 0.        ,
        0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.125     , 0.        , 0.        ,
        0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.125     , 0.        , 0.33333333,
        0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.125     , 0.33333333, 0.33333333,
        0.5       , 0.        , 0.        ],
       [1.        , 0.        , 0.125     , 0.        , 0.        ,
        0.        , 0.        , 0.        ]])

## Exercise 3

In [14]:
def steadystate(A, N = 'none', d =.85, tol = 1e-5, maxiter=1000):
    '''
    Input A is the adjacency matrix - we can find it by using
    for example the function above. Output will be a probability
    density vector for each of the nodes of the graph.
    '''
    # Restrict to relevant section of A
    if N != 'none':
        A = A[:N,:N]
    n = A.shape[0]
    #Initialize random vector for p_t
    p_t = np.random.rand(n)
    #normalize
    p_t = p_t / np.sum(p_t)
    #Initialize distance and iter counters
    pdist = 5
    iter = 0
    while (pdist > tol) & (iter < maxiter):
        pnext = d * A @ p_t + ((1 - d)/ n)
        pdist = la.norm(pnext - p_t)
        p_t = pnext
        print("At iter:", iter, "Distance was", pdist)
        iter += 1
    return(p_t)

Data = readmatfromtext("Data/matrix.txt").toarray()
A = Kmat(Data)

print("Compute steady-state:")
steadystate(A)
    

Compute steady-state:
At iter: 0 Distance was 0.3977708407215281
At iter: 1 Distance was 0.261953778207579
At iter: 2 Distance was 0.1607067540560277
At iter: 3 Distance was 0.1478835252342078
At iter: 4 Distance was 0.12446934452876753
At iter: 5 Distance was 0.1060788318385615
At iter: 6 Distance was 0.09010053996785777
At iter: 7 Distance was 0.07660096951530819
At iter: 8 Distance was 0.06510717662599859
At iter: 9 Distance was 0.05534195470165871
At iter: 10 Distance was 0.04704046090074032
At iter: 11 Distance was 0.03998443880542127
At iter: 12 Distance was 0.033986761947890534
At iter: 13 Distance was 0.028888750244453587
At iter: 14 Distance was 0.024555437100480113
At iter: 15 Distance was 0.020872121677866333
At iter: 16 Distance was 0.017741303392767735
At iter: 17 Distance was 0.015080107891691915
At iter: 18 Distance was 0.012818091706099224
At iter: 19 Distance was 0.010895377950615763
At iter: 20 Distance was 0.0092610712579222
At iter: 21 Distance was 0.007871910569257

array([0.43869011, 0.02171029, 0.02786154, 0.02171029, 0.02171029,
       0.02786154, 0.04585394, 0.39460202])

## Problem 4 Incomplete

In [23]:
def sseigen(A, N='none', d = .85):
    n = len(A[0])
    #Define B matrix
    K = Kmat(A)
    E = np.ones((n, n))
    B = (d * K + ((1 - d) / n) * E)
    #Get eigenvalues
    eigs, eigvecs = la.eig(B)
    p_ss = eigvecs[:,0]/ np.sum(eigvecs[:,0])
    return p_ss

Data = readmatfromtext("Data/matrix.txt").toarray()
A = Kmat(Data)
print(sseigen(A))
print("This isn't right, I think - I should be getting the same SS as in Exercise 3?")

[0.03870928 0.02641669 0.48031139 0.32401996 0.04643401 0.03127529
 0.02641669 0.02641669]
This isn't right, I think - I should be getting the same SS as in Exercise 3?


## Exercise 5

In [6]:
raw = pd.read_csv('Data/ncaa2013.csv')
winners = raw['WINNING_TEAM']
# NOTE THAT THERE IS A SPACE BEFORE LOSING TEAM!!!
losers = raw[' LOSING_TEAM']
names = np.unique(np.concatenate((winners, losers)))
n = len(names)
# Create adjacency matrix (Code from link predictor)
adj = np.zeros((n, n))
for i in raw.index:
    name1, name2 = winners[i], losers[i]
    name1loc = np.where(name1 == names)
    name2loc = np.where(name2 == names)
    '''
    Note - this is where I deviate from the earlier method.
    Now, I have a directed graph, so I only want one of the
    connections to be switched to 1.
    '''
    adj[name2loc, name1loc] = 1
#Grab true adjacency matrix
A = Kmat(adj)
#Get Steady State of Adjacency matrix.
ss = steadystate(A, d=.7)
# Pull rank indices
namerank = np.argsort(ss)
best = names[namerank]
print(f"The 5 best teams are: {best[-5:][::-1]}")
print("You can tell this is a bad algorithm because Notre Dame isn't ranked number 1!")
#np.where('Cleveland St'==names), np.where('Grambling'==names)


At iter: 0 Distance was 0.0369568174458181
At iter: 1 Distance was 0.010036802584170458
At iter: 2 Distance was 0.004232337870560207
At iter: 3 Distance was 0.0020801294153565976
At iter: 4 Distance was 0.0010539100663760831
At iter: 5 Distance was 0.0005419685328260932
At iter: 6 Distance was 0.000279527479519883
At iter: 7 Distance was 0.0001448214653226444
At iter: 8 Distance was 7.541511619685228e-05
At iter: 9 Distance was 3.941263146505431e-05
At iter: 10 Distance was 2.0664193143839017e-05
At iter: 11 Distance was 1.0865213474267182e-05
At iter: 12 Distance was 5.727553454944199e-06
The 5 best teams are: ['Duke' 'Butler' 'Louisville' 'Illinois' 'Indiana']
You can tell this is a bad algorithm because Notre Dame isn't ranked number 1!
